<a href="https://colab.research.google.com/github/markjfannon/SAI-24/blob/main/COMP3008_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Z3 and Python

In Lab 1, you have seen that Z3 can magically solve mathematical problems, and you do not need to think about algorithms.  All you have to do is to formulate the requirements in a declarative language.  However you might have noticed that the SMT2 language (that language used by Z3) is not particularly user-friendly.

Today, you will learn how to use Z3 from a Python program (without the SMT2 language).  The skill you will acquire is not just about this specific combination (Z3 and Python); it is about the concept of using a general-purpose solver such as Z3 from a general-purpose programming language such as Python.

Why Python?  While relatively slow, it is concise and flexible which makes it ideal for working with external libraries.  It is a popular choice in applications of machine learning and other AI tools.

Some of you might not have experience in Python.  This is OK; knowing how to write programs in any procedural language such as Java and some minimal understanding of the Python syntax will be sufficient for the labs/coursework.  Also, learning a bit of Python along the way will certainly be good for your studies and career.

Before proceeding, please run the following command to install the `z3-solver` package:

In [ ]:
!pip install z3-solver

Below is an example of a Python program that encodes a single rule $A \land \lnot B$ and calls Z3 to find the values of $A$ and $B$ that satisfy the rule.

In [ ]:
import z3

s = z3.Solver()

A = z3.Bool('A')
B = z3.Bool('B')
s.add(z3.And(A, z3.Not(B)))

if s.check() == z3.unsat:
    print('unsat')
else:
    print(f'A = {s.model().eval(A)}')
    print(f'B = {s.model().eval(B)}')

Read it carefully.  Note how Boolean constants are defined.  The `add` function is equivalent of `assert` in the Z3 language; it adds an expression to the knowledge base.  Functions `Not`, `And`, etc. are provided by the Z3 library to encode logical expressions.  Function `s.check()` executes the reasoning algorithm of Z3.  The result of the function is `sat` if the knowledge base is satisfiable and `unsat` if it is not.
It can also be `unknown` but this is unlikely to happen with simple examples.  The `s.model().eval` function gives you access to the model found by Z3; it returns the value of the given constant.

## Exercises

1. 	Use the above approach to check satisfiability of the following formula:
	$$
	((A \land \lnot B) \lor (\lnot A \land B)) \land (A \leftrightarrow B) \land \lnot (C \leftrightarrow B)
	$$

In [ ]:
# Your solution to exercise 1



2.  Encode your solution to the last exercise from Lab 1.  To create an integer constant, you will need function `z3.Int` instead of `z3.Bool`.  You can use operators such as `>=`, `+`, etc. to compose Z3 expressions.  All the Z3 library functions are [described here](https://z3prover.github.io/api/html/namespacez3py.html#func-members).

In [ ]:
# Your solution to exercise 2



3.  Study the following Python script.  At what point does the program calculate $x + y$ and compare it to 4?  What exactly happens in line `s.add(x + y >= 4)`?  **It is crucial that you correctly understand the answer.**

In [ ]:
import z3

s = z3.Solver()
x = z3.Int('x')
y = z3.Int('y')
s.add(x + y >= 4)
if s.check() == z3.unsat:
    print('unsat')
else:
    print(f'x = {s.model().eval(x)}')
    print(f'y = {s.model().eval(y)}')

4. A disc jockey (DJ) composes a playlist of 30 tracks. 	The playlist consists of 'slow blocks', i.e. blocks of slow tracks, and 'fast blocks', i.e. blocks of fast tracks. 	The fast and slow blocks alternate.  The DJ wants to follow several rules:
 1. The slow block cannot include more than two tracks.
 2. The fast block has to include at least three tracks.
 3. Tracks 1, 8, 15, 22, ... have to be slow.  (Assume indexing from 1.)
 4. Tracks 6, 11, 16, 21, ... have to be fast.

 Write a Python script that finds the desired structure of the playlist using Z3.  You are expected to use an array of Boolean constants; please don't create 30 variables by hand!  Then use loops to encode the rules and print the solution.

 Hints:
 * You can use conditions such as 'if the $i$th track is slow and the $i+1$th track is slow then...'
 * It might help you a lot to write down those conditions on a piece of paper before encoding them in Python.
 * You will create such conditions for each $i$.
 * A condition can be encoded using implication.

In [ ]:
# Your solution to exercise 4

